<a href="https://colab.research.google.com/github/Melynna/Aula_01/blob/main/ProjetoP21_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import hashlib
import os #Biblioteca de comandos do python. Possivel ler ou escrever arquivos
import datetime #Manipulação de data e hora
import pickle #mantém o controle dos objetos que já serializou, para que referências posteriores ao mesmo objeto não sejam serializadas novamente
from tabulate import tabulate #Biblioteca que permite criar tabelas de forma fácil

# Paths for credentials file
CREDENTIALS_FILE = "credentials.txt"
INVENTORY_FILE = "inventory.txt"

# In-memory data store for objects
inventory = {}

class MLB: #Modelo para criar objeto dentro do programa
  def __init__(self, serial_number, username): #"def" define funções
    self.serial_number = serial_number #"self" indica que esta falando do próprio objeto
    self.status = "checked_in"
    self.checked_in_at = None
    self.last_checked_in_by = None
    self.checked_out_at = None
    self.last_checked_out_by = None

  def check_in(self, username):
    self.checked_in_at = datetime.datetime.now()
    self.status = "checked_in"
    self.last_checked_in_by = username

  def check_out(self, username):
    self.checked_out_at = datetime.datetime.now()
    self.status = "checked_out"
    self.last_checked_out_by = username

def load_credentials():
    if not os.path.exists(CREDENTIALS_FILE):
        open(CREDENTIALS_FILE, 'w').close()  # Cria o arquivo se não existir
    credentials = {}
    with open(CREDENTIALS_FILE, 'r') as file:
        for line in file:
            username, password_hash = line.strip().split(':')
            credentials[username] = password_hash
    return credentials

def save_credentials(username, password_hash):
    with open(CREDENTIALS_FILE, 'a') as file:
        file.write(f"{username}:{password_hash}\n")

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def authenticate_user(credentials):
    username = input("Nome: ").strip() #"strip" remove espaços em branco no começo e no final de uma string
    password = input("Password: ").strip()
    password_hash = hash_password(password)

    if username in credentials and credentials[username] == password_hash:
        print("Login com sucesso!")
        return username
    else:
        print("Nome ou password invalidos.")
        return

def register_user(credentials):
    username = input("Entre com um novo nome de usuario: ").strip()
    if username in credentials:
        print("Usuario ja existente.")
        return

    password = input("Entre com um novo password: ").strip()
    password_hash = hash_password(password)
    save_credentials(username, password_hash)
    print("Usuario registrado com sucesso!")

def load_inventory():
    global inventory
    if not os.path.exists(INVENTORY_FILE) or os.path.getsize(INVENTORY_FILE) == 0:
        open(INVENTORY_FILE, 'w').close()
        return

    with open(INVENTORY_FILE, "rb") as file:  # Use 'rb' para ler em binário
        inventory = pickle.load(file)

def save_inventory():
    with open(INVENTORY_FILE, "wb") as file:
        pickle.dump(inventory, file)

def check_in(username):
    mlb_serial_number = input("Entre com o Serial number da MLB: ").strip()
    input_bin = input("Entre com o Bin: ").strip()
    input_slot = input("Entre com o Bin Slot: ").strip()

    if not input_bin in inventory:
        inventory[input_bin] = {}

    if not input_slot in inventory[input_bin]:
        inventory[input_bin][input_slot] = MLB(mlb_serial_number, username)

    mlb = inventory[input_bin][input_slot]
    mlb.check_in(username)
    save_inventory()

    print(f"MLB '{mlb_serial_number}' checked in por {username}.")

def check_out(username):
    input_bin = input("Entre com o numero do Bin: ").strip()

    if not input_bin in inventory:
        print("Bin esta vazio.")
        return

    input_slot = input("Entre com o numero do slot: ").strip()

    if not input_slot in inventory[input_bin]:
        print("Slot esta vazio.")
        return

    mlb = inventory[input_bin][input_slot]
    mlb.check_out(username)
    save_inventory()

    print(f"Checkout efetuado com sucesso por {username}.")


def show_inventory():
    headers = ["Bin", "Slot", "Serial Number", "Status", "Last Check In Date", "Last Check In By", "Last Check Out Date", "Last Check Out By"]
    data = []

    for mlb_bin, mlb_slots in inventory.items():
        for mlb_slot, mlb in mlb_slots.items():
            data.append([mlb_bin, mlb_slot, mlb.serial_number, mlb.status, mlb.checked_in_at, mlb.last_checked_in_by, mlb.checked_out_at, mlb.last_checked_out_by])

    print(tabulate(data, headers=headers, tablefmt="pretty"))

def main():
    credentials = load_credentials()
    load_inventory()

    print("Bem vindo ao Sistema de controle de MLB")
    print("1. Login")
    print("2. Registro")

    username = {}
    choice = input("Selecione uma opção: ").strip()
    print("\n")

    if choice == "1":
        username = authenticate_user(credentials)
    elif choice == "2":
        register_user(credentials)
        username = authenticate_user(credentials)
    else:
        print("Entrada invalida. Exiting.")
        return

    if username:
        while True:
            print("\nOpções:")
            print("1. Check in MLB")
            print("2. Check out MLB")
            print("3. Checar armezenamento")
            print("4. Sair")

            choice = input("Selecione sua escolha: ").strip()
            print("\n")
            if choice == "1":
                check_in(username)
            elif choice == "2":
                check_out(username)
            elif choice == "3":
                print("Inventory:")
                show_inventory()
            elif choice == "4":
                print("Até!")
                break
            else:
                print("Escolha inválida.")

if __name__ == "__main__":
    main()


Bem vindo ao Sistema de controle de MLB
1. Login
2. Registro
Selecione uma opção: 1


Nome: 123
Password: 123
Login com sucesso!

Opções:
1. Check in MLB
2. Check out MLB
3. Checar armezenamento
4. Sair
Selecione sua escolha: 3


Inventory:
+-----+------+---------------+-------------+----------------------------+------------------+----------------------------+-------------------+
| Bin | Slot | Serial Number |   Status    |     Last Check In Date     | Last Check In By |    Last Check Out Date     | Last Check Out By |
+-----+------+---------------+-------------+----------------------------+------------------+----------------------------+-------------------+
|  1  |  1   |       1       | checked_in  | 2024-11-19 23:14:55.455252 |       123        |                            |                   |
|  1  |  2   |       2       | checked_out | 2024-11-19 23:15:29.942853 |       123        | 2024-11-19 23:20:55.644534 |        123        |
|  1  |  3   |       3       | checked_in  | 2024

In [10]:
import sqlite3

# Função para conectar ao banco de dados
def conectar():
    return sqlite3.connect('clientes.db')

# Função para criar a tabela no banco de dados
def criar_tabela():
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS clientes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        idade INTEGER NOT NULL
    )
    ''')
    conn.commit()
    conn.close()

# Função para inserir um cliente no banco de dados
def inserir_cliente(nome, idade):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO clientes (nome, idade)
    VALUES (?, ?)
    ''', (nome, idade))
    conn.commit()
    conn.close()

# Função para ler todos os clientes no banco de dados
def ler_clientes():
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM clientes')
    clientes = cursor.fetchall()
    conn.close()
    return clientes

# Função para atualizar a idade de um cliente
def atualizar_cliente(id_cliente, nome, idade):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute('''
    UPDATE clientes
    SET nome = ?, idade = ?
    WHERE id = ?
    ''', (nome, idade, id_cliente))
    conn.commit()
    conn.close()

# Função para deletar um cliente
def deletar_cliente(id_cliente):
    conn = conectar()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM clientes WHERE id = ?', (id_cliente,))
    conn.commit()
    conn.close()

# Exemplo de uso do programa CRUD
if __name__ == '__main__':
    # Criar a tabela se não existir
    criar_tabela()

    # Inserir alguns clientes
    inserir_cliente('João', 30)
    inserir_cliente('Maria', 25)
    inserir_cliente('Carlos', 35)

    # Ler e mostrar todos os clientes
    print("Clientes antes da atualização:")
    clientes = ler_clientes()
    for cliente in clientes:
        print(cliente)

    # Atualizar a idade de um cliente
    atualizar_cliente(1, 'João Silva', 32)

    # Ler e mostrar todos os clientes novamente após atualização
    print("\nClientes após a atualização:")
    clientes = ler_clientes()
    for cliente in clientes:
        print(cliente)

    # Deletar um cliente
    deletar_cliente(2)

    # Ler e mostrar todos os clientes novamente após exclusão
    print("\nClientes após a exclusão:")
    clientes = ler_clientes()
    for cliente in clientes:
        print(cliente)


Clientes antes da atualização:
(1, 'João', 30)
(2, 'Maria', 25)
(3, 'Carlos', 35)

Clientes após a atualização:
(1, 'João Silva', 32)
(2, 'Maria', 25)
(3, 'Carlos', 35)

Clientes após a exclusão:
(1, 'João Silva', 32)
(3, 'Carlos', 35)
